In [20]:
from s3prl.nn import S3PRLUpstream
import torch
import pytorch_lightning as pl

In [5]:
upstream = 

Downloading: https://huggingface.co/s3prl/converted_ckpts/resolve/main/audio_base_ls.pt
Destination: /home/eernst/.cache/s3prl/download/a303ec55138fdcb6c5359385c2a14b7045bb5b2c74b14239b32791d77424e8dd.audio_base_ls.pt
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 696M/696M [00:35<00:00, 20.8MB/s]


In [52]:
class S3PRLUpstreamMLPDownstrem(pl.LightningModule):
    def __init__(self, state, upstream='data2vec', layer=-1, hidden_layers=2, hidden_dim=128):
        super().__init__()
        self.upstream = S3PRLUpstream(upstream)
        self.mapping = state['class_map']
        upstream_dim = self.upstream.hidden_sizes[0]
        layer_dims = [upstream_dim] + [hidden_dim]*hidden_layers
        self.net = torch.nn.Sequential(*[torch.nn.Sequential(torch.nn.Linear(dim_in, dim_out), torch.nn.ReLU()) for dim_in, dim_out in zip(layer_dims[:-1],layer_dims[1:])])
        if isinstance(layer, int):
            layer = [layer]
        if layer == 'all':
            layer = list(range(len(self.upstream.hidden_sizes)))
        self.avg_weights = torch.nn.Parameter(torch.ones(len(layer),))
        self.layer = layer
        self.out_layer = torch.nn.Linear(layer_dims[-1],len(self.mapping))

    def forward(self, x):
        with torch.no_grad():
            hidden, last = upstream(x['wav'], wavs_len=x['wavs_len'])
        hidden = torch.stack(hidden).transpose(0,1)
        print(hidden.shape)
        w = torch.nn.functional.softmax(self.avg_weights, dim=0)
        avg_hidden = torch.sum(hidden[:,self.layer]*w[None,:,None,None],dim=1)
        y = self.out_layer(self.net(avg_hidden))
        return y

    def training_step(self, batch, batch_idx):
        yhat=self(batch)
        y=batch['class_id']
        return torch.nn.functional.cross_entropy(yhat,y)

    def validation_step(self, batch):

    def configure_optimizers(self):
        return torch.optim.Adam(
        

In [31]:
import joblib

state=joblib.load('../speech_hypertuning/experiments/experiment_lr/test_load_dataset/state.pkl')

In [53]:
model = S3PRLUpstreamMLPDownstrem(state, layer='all')

In [54]:
model({'wav':torch.randn((2,32000)),'wavs_len':torch.tensor([32000,16000])}).shape

torch.Size([2, 13, 100, 768])


torch.Size([2, 100, 1251])

In [16]:
hidden, last = upstream(torch.randn((2,32000)), wavs_len=torch.tensor([32000,16000]))

In [19]:
hidden[0].shape

torch.Size([2, 100, 768])

In [26]:
hidden = torch.stack(hidden).transpose(0,1)

In [28]:
hidden[:,[2,3,5]].shape

torch.Size([2, 3, 100, 768])

In [ ]:
BS,L,T,D
x,L,x,x